# Python for Data Analysis v2 | Notes_ Chapter_6 数据加载、存储与文件格式

本人以简书作者 SeanCheney 系列专题文章并结合原书为学习资源，记录个人笔记，仅作为知识记录及后期复习所用，原作者地址查看 [简书 SeanCheney](https://www.jianshu.com/u/130f76596b02)，如有错误，还望批评指教。——ZJ


>原作者：SeanCheney | [利用 Python 进行数据分析·第2版》第5章  pandas 入门](https://www.jianshu.com/p/161364dd0acf) | 來源：简书

>[Github:wesm](https://github.com/wesm/pydata-book) | [Github:中文 BrambleXu](https://github.com/BrambleXu/pydata-notebook)|
简书:[利用 Python 进行数据分析·第2版](https://www.jianshu.com/c/52882df3377a)

环境： Python  3.6 

---


访问数据是使用本书所介绍的这些工具的第一步。我会着重介绍pandas的数据输入与输出，虽然别的库中也有不少以此为目的的工具。

输入输出通常可以划分为几个大类：读取文本文件和其他更高效的磁盘存储格式，加载数据库中的数据，利用Web API操作网络资源。

## 6.1 读写文本格式的数据

pandas提供了一些用于将表格型数据读取为DataFrame对象的函数。表6-1对它们进行了总结，其中read_csv和read_table可能会是你今后用得最多的。

